# Problem 1 - The Conditional Logit Model

In [44]:
from choice_learn.datasets import load_modecanada
from choice_learn.data import ChoiceDataset
from choice_learn.models import ConditionalLogit
import pandas as pd

## 1a) Inspecting transportation choice dataset

In [2]:
transport_df = load_modecanada(as_frame=True)
print(f"Dataset shape: {transport_df.shape}")
display(transport_df.head(8))

Dataset shape: (15520, 11)


,case,alt,choice,dist,cost,ivt,ovt,freq,income,urban,noalt
0,1,train,0,83,28.25,50,66,4,45.0,0,2
1,1,car,1,83,15.77,61,0,0,45.0,0,2
2,2,train,0,83,28.25,50,66,4,25.0,0,2
3,2,car,1,83,15.77,61,0,0,25.0,0,2
4,3,train,0,83,28.25,50,66,4,70.0,0,2
5,3,car,1,83,15.77,61,0,0,70.0,0,2
6,4,train,0,83,28.25,50,66,4,70.0,0,2
7,4,car,1,83,15.77,61,0,0,70.0,0,2


Each row represents one alternative in a choice situation.

- `case`: ID for each choice situation. In this scenario, a person choosing between some viable alternative modes of transport

- `choice`: ==1 for the chosen alternative. ==0 otherwise.

In [52]:
sorted(transport_df["alt"].unique())

['air', 'bus', 'car', 'train']

There are four unique alternatives represented in the dataset. 'train', 'car', 'bus', and 'air'.

case 1 had an option between two alternatives. They could either take the train or a car, and they chose to take a car.

## 1b) Converting to a ChoiceDataset

Every ChoiceDataset must be created with a few attributes:


- A column to uniquely identify each alternative
- A column to uniquely identify each choice situation
- A column to uniquely identify the choice made in each choice situation

And then we partition the additional covariates into:

- Features relating to the decision maker (in this example, each traveller)
- Features relating to the choice alternatives (in this example, the modes of transport)

Finally, we explicitly tell the ChoiceDataset object that the chosen alternative is being indicated by a binary variable.

In [4]:
canada_dataset = ChoiceDataset.from_single_long_df(
df=transport_df,
items_id_column="alt", # identifies each alternative
choices_id_column="case", # identifies each choice situation
choices_column="choice", # indicates which was chosen
shared_features_columns=["income"], # traveler characteristics
items_features_columns=["cost", "freq", "ovt", "ivt"], # alternative attributes
choice_format="one_zero"
)
print(canada_dataset.summary())

%=====================================================================%
%%% Summary of the dataset:
%=====================================================================%
Number of items: 4
Number of choices: 4324
%=====================================================================%
 Shared Features by Choice:
 1 shared features
 with names: (['income'],)


 Items Features by Choice:
4 items features 
 with names: (['cost', 'freq', 'ovt', 'ivt'],)
%=====================================================================%



There are:

- **4 unique choice alternatives** (modes of transport)

- **4324 unique choice situations** (travellers)

## 1c) Specification of the individual's utility function

$$
U_{ij}
=
\beta^{inter}_{j}
+\beta^{cost}\cdot cost_{j}
+\beta^{freq}\cdot freq_{j}
+\beta^{ovt}\cdot ovt_{j}
+\beta^{ivt}_{j}\cdot ivt_{j}
+\beta^{income}_{j}\cdot income_{i}
+\xi_{ij}
+\epsilon_{ij}
$$

- Constant intercept term for each alternative: $\beta^{inter}_{j}$

- Features from each alternative:
    - $\beta^{cost}$: cost of transport
    - $\beta^{freq}$: how frequently the transport mode arrives
    - $\beta^{ovt}$: How much time commuting is spent outside the vehicle

- Features specific to each decision maker:
    - $\beta^{income}_{j}$: Income of the individual
    - $\beta^{ivt}_{j}$: How much of commuting time is spent inside the vehicle

- $\xi_{ij}$: Unobserved (by the econometrician) characteristics.

- $\epsilon_{ij}$: Errors. Assumed to follow a Type-I exttreme value distribution.

We assume that the unobserved characteristics are not confounding our analysis.

We account for heterogeneous effects of $ivt_{j}$ because agents may have different preferences on being in different vehicles. For instance, trains may be less comfortable that driving one's own car. So we may expect an equal length journey taken by train to deliver more disutility than time spent driving a car.

Our assumed utility specification is entirely linear in all parameters.

## 1d) Fitting the Conditional Logit Model

First, we initialise all of the model parameters.

`choice-learn` does not use pasty formulae (e.g. statsmodels). Instead, we need to follow the following workflow:

- Choose an optimiser

- Add the shared coefficients (i.e. those that don't vary across alternatives or individuals)

    - Each coefficient must match the name of a feature in the ChoiceDataset object
    - Indicate for which choice alternatives the shared coefficient will applied *[items_indexes]*

- Add the heterogeneous coefficients
    - Indicate for which choice indices a new parameter should be estimated *[items_indexes]*


Note that the items_indexes argument does something different when we are adding shared coefficients vs heterogeneous coefficients.

Then, we fit the model.

In [5]:
# Initialise the model

# Choose the optimiser
transport_model = ConditionalLogit(optimizer="lbfgs")

# Alternative-specific intercepts
# Exclude one index to avoid multicollinearity
transport_model.add_coefficients("intercept", items_indexes=[1, 2, 3])

# Parameters shared across all modes
transport_model.add_shared_coefficient("cost", items_indexes=[0, 1, 2, 3])
transport_model.add_shared_coefficient("freq", items_indexes=[0, 1, 2, 3])
transport_model.add_shared_coefficient("ovt",  items_indexes=[0, 1, 2, 3])

# Alternative-specific parameters (one per mode)
transport_model.add_coefficients("ivt", items_indexes=[0, 1, 2, 3])

# Alternative-specific coefficients on an individual-specific variable:
# Omit the base index to avoid multicollinearity
transport_model.add_coefficients("income", items_indexes=[1, 2, 3])

Using L-BFGS optimizer, setting up .fit() function


In [6]:
# Estimation of the model
# Pass in the ChoiceDataset, not the pd.DataFrame
history = transport_model.fit(canada_dataset, get_report=True, verbose=2)

Using L-BFGS optimizer, setting up .fit() function


The optimiser converged upon a soluition in 258 iterations. The model weights and loss are stored in the fitted model object.

## 1e) Model outputs

In [7]:
transport_model.trainable_weights

[<tf.Variable 'beta_intercept:0' shape=(1, 3) dtype=float32, numpy=array([[1.1081554, 2.7675772, 3.2506676]], dtype=float32)>,
 <tf.Variable 'beta_cost:0' shape=(1, 1) dtype=float32, numpy=array([[-0.00746423]], dtype=float32)>,
 <tf.Variable 'beta_freq:0' shape=(1, 1) dtype=float32, numpy=array([[0.07528259]], dtype=float32)>,
 <tf.Variable 'beta_ovt:0' shape=(1, 1) dtype=float32, numpy=array([[-0.04007844]], dtype=float32)>,
 <tf.Variable 'beta_ivt:0' shape=(1, 4) dtype=float32, numpy=
 array([[ 0.00029401, -0.01162978, -0.01573393, -0.00622123]],
       dtype=float32)>,
 <tf.Variable 'beta_income:0' shape=(1, 3) dtype=float32, numpy=array([[-0.06451878, -0.02576683, -0.03880461]], dtype=float32)>]

In [8]:
print("The average neg-loglikelihood is:", transport_model.evaluate(canada_dataset).numpy())
print("The total neg-loglikelihood is:", transport_model.evaluate(canada_dataset).numpy()*len(canada_dataset))

The average neg-loglikelihood is: 0.6083679
The total neg-loglikelihood is: 2630.5828857421875


And we can obtain a full model report:

In [9]:
transport_model.report

,Coefficient Name,Coefficient Estimation,Std. Err,z_value,P(.>z)
0,beta_intercept_0,1.108155,0.271243,4.085472,4.398728e-05
1,beta_intercept_1,2.767577,0.176806,15.653187,0.000000e+00
2,beta_intercept_2,3.250668,0.246447,13.190132,0.000000e+00
3,beta_cost,-0.007464,0.002941,-2.537747,1.115686e-02
4,beta_freq,0.075283,0.004203,17.913317,0.000000e+00
5,beta_ovt,-0.040078,0.002365,-16.949123,0.000000e+00
6,beta_ivt_0,0.000294,0.004265,0.068935,9.450415e-01
7,beta_ivt_1,-0.011630,0.001637,-7.103394,1.217249e-12
8,beta_ivt_2,-0.015734,0.000991,-15.879060,0.000000e+00
9,beta_ivt_3,-0.006221,0.000682,-9.116179,0.000000e+00


In [53]:
# So that we know which index refers to which mode of transport
sorted(transport_df["alt"].unique())

['air', 'bus', 'car', 'train']

The coefficent on transport cost is negative. This suggests that higher costs reduce the probability of choosing that mode of transport. This makes economic sense, an inverse relationship between costs and demand.


If we look at all the intercept terms and note that the 'air' is the baseline (since it is at index 0):

- train travel has the greatest baseline utility, followed by car, then bus, then air.

Finally, if we look at the coefficients on income by mode of transport, we can see that:

- Relative to air travel, all other modes of transport have a more negative relationship between income and utility. This suggests that higher incomes have a greater positive effect on utility for air than for all other modes of transport. Income has the most negative effect on bus travel. This makes sense as buses in Canada are a cheaper option.

## 1f) Recovering elasticities

THe logit model has closed form solutions when the errors are Type-I (Gumbel) distributed. This allows us to recover exact forms for the elasticities from the estimated model.

Choice probabilities in the model are given by the usual log-odds ratio:

$$
P_{ij}
=
\Pr(y_i=j)
=
\frac{\exp\!\left(V_{ij}\right)}
{\sum\limits_{k\in\mathcal{J}}\exp\!\left(V_{ik}\right)}
$$

Where $V_{ij}$ is the inclusive value of alternative j to agent i, and it is given by:

$$
V_{ij}
=
\beta^{inter}_{j}
+\beta^{cost}\,cost_{ij}
+\beta^{freq}\,freq_{ij}
+\beta^{ovt}\,ovt_{ij}
+\beta^{ivt}_{j}\,ivt_{ij}
+\beta^{income}_{j}\,income_{i}
$$

The own-price elasticity of demand can then be computed as:

$$
\frac{\partial P_{ij}}{\partial cost_{ij}}
=
\beta^{cost}\,P_{ij}\left(1-P_{ij}\right)
$$

$$
\eta_{jj,\;cost}
=
\frac{\partial P_{ij}}{\partial cost_{j}}\cdot \frac{cost_{j}}{P_{ij}}
=
\beta^{cost}\,cost_{j}\left(1-P_{ij}\right)
$$

We want to compute the mean own-price elasticity of demand for cars:

### Recover the parameter on cost/price

In [35]:
# Recover the beta_cost parameter from the model
# It is loaded as a lists of lists, so we need to unpack the object to a float
beta_cost = transport_model.trainable_weights[1].numpy()[0][0]
beta_cost

-0.007464232

### Recover the average probability of choosing a car

In [43]:
# We can compute and store the choice probabilities
probabilities = transport_model.predict_probas(canada_dataset)
probabilities

<tf.Tensor: shape=(4324, 4), dtype=float32, numpy=
array([[0.        , 0.        , 0.8689874 , 0.13100392],
       [0.        , 0.        , 0.8363469 , 0.16364476],
       [0.        , 0.        , 0.9018492 , 0.09814065],
       ...,
       [0.12176416, 0.        , 0.83322936, 0.0449619 ],
       [0.        , 0.        , 0.9908673 , 0.00888368],
       [0.        , 0.        , 0.94508195, 0.0548813 ]], dtype=float32)>

The probabilities are output as a 4324 x 4 tensor. Each column corresponds to one of the choice alternatives, and the probabilities across each row sum to 1.

The 'car' alternative is at index 2, so we slice the third column from the tensor and compute the mean across all choice situations.

In [54]:
car_mean_prob = float(probabilities[:, 2].numpy().mean())
car_mean_prob

0.5117910504341125

On average, travel by car has a 51% probability of being chosen.

### Recover the average cost of travel by car

In [55]:
# Compute the mean car cost
car_mean_cost = transport_df.loc[transport_df["alt"] == "car", "cost"].mean()
car_mean_cost

63.76371877890842

In [57]:
# Compute the own-price elasticity for cars
car_mean_elasticity = beta_cost * car_mean_cost * (1 - car_mean_prob)
car_mean_elasticity

-0.23236167536116545

Cars have a negative mean own-price elasticity of $-0.23236167536116545$. This suggests that a percentage point increase in cost leads to a reduction in demand for cars by roughly 23%.